# N-Dimensiaonal

Para explorar datos de mayor dimensión y las relaciones entre los atributos de datos, cargaremos en el archivo `Diabetes.csv`. Para más información [Kaggle](https://www.kaggle.com/uciml/pima-indians-diabetes-database). 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
df_original = pd.read_csv("Diabetes.csv")
df_original.head()

In [ ]:
cols = [c for c in df_original.columns if c not in ["Pregnancies", "Outcome"]]
df = df_original.copy()
df[cols] = df[cols].replace({0: np.NaN})
df.head()

In [ ]:
df.info()

# Observando los datos

In [ ]:
df.describe()

# Matriz Dispersión

Este es un trazador de líneas útil ... Hay que tener en cuenta que sólo funciona con datos numéricos. Si desea incluir datos categóricos, se deberá convertir las categorías en etiquetas numéricas.

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(7, 7));

In [ ]:
df2 = df.dropna()
colors = df2["Outcome"].map(lambda x: "#44d9ff" if x else "#f95b4a")
pd.plotting.scatter_matrix(df2, figsize=(7,7), color=colors);

El gráfico de dispersión proporciona los histogramas para las distribuciones a lo largo de la diagonal y también muchos gráficos de dispersión 2D fuera de la diagonal. No es que esta sea una matriz simétrica, así que solo miro la diagonal y debajo de ella normalmente. Se puede ver que algunas variables tienen mucha dispersión y algunas están correlacionadas (es decir, hay una dirección en su dispersión). Lo que nos lleva a...

# Gráficos de correlación

¡Para cuantificar fácilmente qué variables/atributos están correlacionados con otros!

In [ ]:
df.corr()

In [ ]:
sb.heatmap(df.corr());

In [ ]:
sb.heatmap(df.corr(), annot=True, cmap="viridis", fmt="0.2f");

Se puede ver que esta también es una matriz simétrica. Pero de inmediato permite señalar los atributos más correlacionados y anticorrelacionados. Algunos pueden ser simplemente de sentido común (Embarazos versus Edad, por ejemplo), pero otros pueden darnos una idea real de los datos.


# Histogramas 2D

Para el resto de esta sección, vamos a utilizar un conjunto de datos diferente que contiene más datos.

Útil cuando tienes muchos *muchos* datos. [Vea aquí la API](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist2d.html)

In [ ]:
df2 = pd.read_csv("height_weight.csv")
df2.info()
df2.describe()

In [ ]:
plt.hist2d(df2["height"], df2["weight"], bins=20, cmap="magma")
plt.xlabel("Height")
plt.ylabel("Weight");

# Gráficos de contorno

Un poco difícil obtener información del histograma 2Dimensiones. Demasiado ruido en la imagen. ¿Y si se intenta un diagrama de contorno? Se tendra que almacenar.  [Vea aquí la API](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.contour.html)

In [ ]:
hist, x_edge, y_edge = np.histogram2d(df2["height"], df2["weight"], bins=20)
x_center = 0.5 * (x_edge[1:] + x_edge[:-1])
y_center = 0.5 * (y_edge[1:] + y_edge[:-1])

plt.contour(x_center, y_center, hist, levels=4)
plt.xlabel("Height")
plt.ylabel("Weight");

¡Parece que es igual de ruidoso con el gráfico de contorno! En general, para histogramas 2D y gráficos de contorno, tenga una gran cantidad de datos. ¡Simplemente no se tiene suficientes datos para obtener resultados fluidos!

# Parcelas KDE

Si tan solo pudiéramos suavizar los datos nosotros mismos. [Seaborn](https://seaborn.pydata.org/generated/seaborn.kdeplot.html)

In [ ]:
sb.kdeplot(df2["height"], df2["weight"], cmap="viridis", bw=(2, 20));
plt.hist2d(df2["height"], df2["weight"], bins=20, cmap="magma", alpha=0.3);

In [ ]:
sb.kdeplot(df2["height"], df2["weight"], cmap="magma", shade=True);

# En defensa de la simplicidad

A veces no se necesita ir de lujo. Al hacer una pausa, hacer un balance de lo que se quiere ver, un diagrama de dispersión normalmente es bastante informativo y muy rápido de trazar.

In [ ]:
m = df2["sex"] == 1
plt.scatter(df2.loc[m, "height"], df2.loc[m, "weight"], c="#16c6f7", s=1, label="Male")
plt.scatter(df2.loc[~m, "height"], df2.loc[~m, "weight"], c="#ff8b87", s=1, label="Female")
plt.xlabel("Height")
plt.ylabel("Weight")
plt.legend(loc=2);

## Recuerda 
![Recuerda](https://t4.ftcdn.net/jpg/00/28/30/47/240_F_28304775_VQ4dh7jHcwQOV3cUREVjAmhMkddJV9hr.jpg)

* Cuando se está visualizando datos, comienza de manera simple y agregue complejidad según parezcan indicar los datos. No tiene sentido perder el tiempo profundizando mucho cuando no es necesario.

* **Los gráficos no demuestran nada pero ayudan a entender los datos**

# Tratamiento de puntos con probabilidad

Usando la biblioteca ChainConsumer ([ejemplos aquí](https://samreay.github.io/ChainConsumer/examples/index.html)). 

ChainConsumer no es un paquete estándar de anaconda, por lo que deberá ejecutar lo siguiente para instalarlo.

`pip install chainconsumer`

In [ ]:
params = ["height", "weight"]
male = df2.loc[m, params].values
female = df2.loc[~m, params].values
male.shape

In [ ]:
from chainconsumer import ChainConsumer
c = ChainConsumer()
c.add_chain(male, parameters=params, name="Male", kde=1.0, color="b")
c.add_chain(female, parameters=params, name="Female", kde=1.0, color="r")
c.configure(contour_labels="confidence", usetex=False)
c.plotter.plot(figsize=2.0);

In [ ]:
c.plotter.plot_summary(figsize=2.0);